# Script for computing spacings

Reference: [Nikki Loppi thesis](https://spiral.imperial.ac.uk/handle/10044/1/73888)


$$Re = \frac{\rho U}{\nu}$$ 

In [4]:
Re_x = 1.2e6 # From Loppi thesis
U_inf = 1    # Set for ease of computations
rho = 1      # Set for ease of computations

# Submarine lengths
# In Fidelity Pointwise, find these with Ctrl+Shift+MMB
L_x_all  = 14.29166
L_x_sail = 1.208333
L_x_stern = 0.5

# Solver 
order = 5    # As per Sambit's benchmarking paper, order 5 is optimal, while order 6 sees performance drops on H100s and PVCs.



In [6]:
nu_all = rho*U_inf*L_x_all/Re_x
Re_x_sail = rho*U_inf*L_x_sail/nu_all
Re_x_stern = rho*U_inf*L_x_stern/nu_all

print("Set in PyFR *.ini file [constants] nu = ", nu_all)
print("Sail Reynolds number: Re = ", Re_x_sail)
print("Stern Reynolds number: Re = ", Re_x_stern)

Set in PyFR *.ini file [constants] nu =  1.1909716666666667e-05
Sail Reynolds number: Re =  101457.7452864118
Stern Reynolds number: Re =  41982.52687231574


## Loppi's $y^+$

### From Loppi's thesis ...
... The boundary layer blocks were generated such that the first P = 4 Gauss-Legendre solution point is at approximately

$$z^+ = \frac{u^\tau\ z}{\mu} = 2.5$$
 with $u\tau = \sqrt{\tau_w}$ being the wall-shear stress reported in reference: Posa and E. Balaras, “A numerical investigation of the wake of an axisym- metric body with appendages,” Journal of Fluid Mechanics, vol. 792, pp. 470– 498, 2016 ...
 

## Low-order $y^+$ spacing

Reference: https://www.cadence.com/en_US/home/tools/system-analysis/computational-fluid-dynamics/y-plus.html

Assumption: Flat-plate boundary layer theory from Frank M. White's Fluid Mechanics 5th edition, page 467.

In [7]:
print("Loppi used y⁺:", 2.5)
y_plus = 2.5

# Overall
C_f = 0.026/Re_x**(1/7)
tau_wall = C_f*0.5*rho*U_inf**2
U_fric = (tau_wall/rho)**0.5
delta_y_all = y_plus*nu_all/U_fric
print("Complete Submarine: ", delta_y_all)
# Sail
C_f = 0.026/Re_x_sail**(1/7)
tau_wall = C_f*0.5*rho*U_inf**2
U_fric = (tau_wall/rho)**0.5
delta_y_sail = y_plus*nu_all/U_fric

print("Sail: ", delta_y_sail)

C_f = 0.026/Re_x_stern**(1/7)
tau_wall = C_f*0.5*rho*U_inf**2
U_fric = (tau_wall/rho)**0.5
delta_y_stern = y_plus*nu_all/U_fric
print("Stern: ", delta_y_stern)


Loppi used y⁺: 2.5
Complete Submarine:  0.0007097362607123966
Sail:  0.0005949241005704607
Stern:  0.0005585845845006936


## High-order $y^+$ spacing

Assuming we use hexahedral elements [Gauss-Legendre quadrature points](https://en.wikipedia.org/wiki/Gauss%E2%80%93Legendre_quadrature), the high-order element may be chosen to be larger than the low-order spacing we just got. Rule of thumb is to multiply the obtained low-order $y^+$ by the element size (anyways we use Flat-plate assumption and these numbers are best obtained after a few simulation runs iteratively). 

In [11]:
# Keeping first point location by Gauss-Legendre quadrature in one place
# Get the numbers from: https://pomax.github.io/bezierinfo/legendre-gauss.html

first_point_loc = {0: 0.0, 1: 0.57735, 2: 0.774597, 3: 0.861136, 4: 0.90618, 5:0.9324695142031521, 6:0.9491079123427585}

Normalised_cell_height_multiplier = [2/(1-x) for x in first_point_loc.values()]
print(Normalised_cell_height_multiplier)

[2.0, 4.7320477936827166, 8.872996366507987, 14.402580942504898, 21.317416329140904, 29.616253702314577, 39.29883980138545]


In [12]:
s_all = delta_y_all*Normalised_cell_height_multiplier[order]
s_sail = delta_y_sail*Normalised_cell_height_multiplier[order]
s_stern = delta_y_stern*Normalised_cell_height_multiplier[order]
ar = 5
print("y⁺ spacing: ", s_all, s_sail, s_stern)
print("Aspect ratio: ", ar)
print("x⁺ spacing: ", s_all*ar, s_sail*ar, s_stern*ar)

y⁺ spacing:  0.02101972915899042 0.01761942309611608 0.016543182768774516
Aspect ratio:  5
x⁺ spacing:  0.1050986457949521 0.0880971154805804 0.08271591384387258


## Free-shear region

In the free shear region, the cell size increases as $$\eta = l^{3}* Re^{-3/4},$$ where $l$ is the total length of submarine.

In [13]:
eta_all = y_plus*L_x_all* Re_x**(-3/4)*Normalised_cell_height_multiplier[order]
eta_sail = y_plus*L_x_sail* Re_x_sail**(-3/4)*Normalised_cell_height_multiplier[order]
eta_stern = y_plus*L_x_stern* Re_x_stern**(-3/4)*Normalised_cell_height_multiplier[order]

print("eta: ", eta_all, eta_sail, eta_stern)


eta:  0.02918548210123611 0.01573775796033254 0.012622305979074502
